# 1) Using the DataProcessor class to preprocess the data

Preprocessing can basically be done with 2 function calls: .remove_columns() and .preprocess_data().

However the class also includes more fine-grained functionality (see code).

In [1]:
import os


In [2]:
# Read the sample file
data_dir = r"/Users/xingyuehuang/Downloads/Tradeteq-adversarial-attacks-on-credit-score-main 2"
co_file = os.path.join(data_dir, "client_start_folder","Co_600K_Jul2019_6M.pkl")


Create a DataProcessor object which loads the data from the right file.

In [3]:
from processing.DataProcessor import DataProcessor
data_proc = DataProcessor(co_file)

Define columns to be removed

In [4]:
zero_info_features = ["CompanyId", "CompanyNumber","CompanyName","imd"]
only_one_value_features = ["Filled", "LimitedPartnershipsNumGenPartners", "LimitedPartnershipsNumLimPartners",\
                          "Status20190701","CompanyStatus"]
complicated_features = ["RegAddressAddressLine1", "RegAddressAddressLine2", "RegAddressCareOf", "RegAddressCounty", \
                        "RegAddressPOBox", "RegAddressPostCode", "RegAddressPostTown","oa11", "PreviousName_1CompanyName"]
to_remove_cols = zero_info_features+only_one_value_features+complicated_features

In [5]:
data_proc.remove_columns(to_remove_cols)

Define columns to be converted to numerical/string type

In [6]:
to_num_cols = ["AccountsAccountRefDay", "AccountsAccountRefMonth", "oac1"]
to_str_cols = ["ru11ind"]

Define the date-Data processing: [["NewDuration Name", "Post Name", "Prev Name"],...]

In [7]:
date_convert=[["dAccountsTimeGap","dAccountsNextDueDate","dAccountsLastMadeUpDate"],\
              ["dConfStmtTimeGap","dConfStmtNextDueDate","dConfStmtLastMadeUpDate"],\
              ["dReturnsTimeGap","dReturnsNextDueDate","dReturnsLastMadeUpDate"]]

In [8]:
X_train, X_test, y_train, y_test = data_proc.preprocess_data(to_num_cols, to_str_cols, date_convert)

In [9]:
import numpy as np
import collections
print(collections.Counter(y_train))
print(collections.Counter(y_test))

Counter({0: 479022, 1: 978})
Counter({0: 119782, 1: 218})


# 2) Evaluating models with the ModelEvaluator class

The ModelEvaluator class is abstract, so that it is general enough to work for different types of models. You might have to build your child class if the syntax differs (see SKEvaluator example)

In [10]:
# import xgboost as xgb
# from ModelEvaluationTools.SKEvaluator import SKEvaluator

In [11]:
# xg = xgb.XGBClassifier(learning_rate=0.3, max_depth=10, subsample=0.5, objective='binary:logistic', verbosity=3)

Create instance of RFEvaluator, a child of the ModelEvaluator class

In [12]:
# xg_model = SKEvaluator(xg, 'xg')

In [13]:
# xg_model.fit(X_train, y_train)

In [14]:
# test_auc = xg_model.evaluate(X_test, y_test)

In [15]:
# saves model to fitted_models folder
# xg_model.save_model()

Now we try the artificial neural network:

In [16]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,BatchNormalization
import keras
import tensorflow as tf
from keras import optimizers

In [17]:
(np.isnan((X_train)).any()).any()

False

In [18]:
# define the keras model
print('Building Neural Network model...')
opt = optimizers.Adam(lr = 0.0001) # hyper parameter
model = Sequential()
model.add(BatchNormalization())
# Try some weight initialization, with different weight
model.add(Dense(10, input_dim=X_train.shape[1],
                kernel_initializer='normal',
                #kernel_regularizer=regularizers.l2(0.02),
                activation="relu"))
model.add(Dense(4,activation = "relu"))
#model.add(Dense(10,
                #kernel_regularizer=regularizers.l2(0.02),
#                activation="relu"))
#model.add(Dense(10,activation = "relu"))
model.add(Dense(1,activation="sigmoid"))

Building Neural Network model...


In [19]:
METRICS = [
     tf.keras.metrics.AUC()
]

In [20]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)

In [41]:
history = model.fit(X_train, y_train, epochs=5, validation_data = (X_test,y_test),batch_size=500,shuffle = True)# change test size hyperparameter
# Change the batch_size to 2000?  callback 

Epoch 1/5
960/960 [==============================] - 2s 2ms/step - loss: 0.0102 - auc: 0.8840 - val_loss: 0.0096 - val_auc: 0.8744
Epoch 2/5
960/960 [==============================] - 2s 2ms/step - loss: 0.0103 - auc: 0.8866 - val_loss: 0.0096 - val_auc: 0.8729
Epoch 3/5
960/960 [==============================] - 2s 2ms/step - loss: 0.0102 - auc: 0.8843 - val_loss: 0.0096 - val_auc: 0.8760
Epoch 4/5
960/960 [==============================] - 2s 2ms/step - loss: 0.0103 - auc: 0.8857 - val_loss: 0.0096 - val_auc: 0.8726
Epoch 5/5
960/960 [==============================] - 2s 2ms/step - loss: 0.0102 - auc: 0.8880 - val_loss: 0.0096 - val_auc: 0.8669


In [42]:
from sklearn.metrics import roc_curve
y_pred_keras =model.predict_proba(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [43]:
print(auc_keras)

0.9375111919681611


In [44]:
print(np.unique(y_pred_keras))

[0.0000000e+00 9.1636450e-38 2.2642820e-36 ... 3.7701672e-01 4.1204277e-01
 4.5231435e-01]


200 batchsize really good